In [4]:
# MeCabとIPAdicのインストール
!apt-get -y install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3 ipadic

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libmecab-dev is already the newest version (0.996-14build9).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-3).
mecab is already the newest version (0.996-14build9).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [5]:
# rouge-scoreライブラリのインストール
!pip install rouge-score

In [1]:
import ipadic
import MeCab
from collections import defaultdict
import pandas as pd
from rouge_score import rouge_scorer, scoring
import os
from typing import Union, List
import csv

pd.options.display.precision = 3

# --- 1. ROUGE計算のためのヘルパー関数 ---

def convert_words_to_ids(predictions: List[str], references: List[str]):
    word2id = defaultdict(lambda: len(word2id))

    pred_ids = [" ".join([str(word2id[w]) for w in p.split()]) for p in predictions]
    ref_ids = [" ".join([str(word2id[w]) for w in r.split()]) for r in references]

    return pred_ids, ref_ids


def compute_rouge(predictions: List[str], references: List[str]):
    rouge = rouge_scorer.RougeScorer(
        rouge_types=["rouge1", "rouge2", "rougeL"], use_stemmer=False
    )
    aggregator = scoring.BootstrapAggregator()

    pred_ids, ref_ids = convert_words_to_ids(predictions, references)

    for pred, ref in zip(pred_ids, ref_ids):
        aggregator.add_scores(rouge.score(ref, pred))

    scores = aggregator.aggregate()
    return {k: v.mid for k, v in scores.items()}


# --- 2. データ読み込み（MeCab 分かち書き） ---

def load_data(input_data: Union[str, List[str]], tagger: MeCab.Tagger) -> List[str]:
    data_list = []

    # ファイル名として渡された場合
    if isinstance(input_data, str):
        if os.path.exists(input_data):
            print(f"📄 ファイル '{input_data}' を読み込みます")
            with open(input_data, "r", encoding="utf-8") as f:
                data_list = [line.strip() for line in f if line.strip()]
        else:
            print("💡 入力文字列として扱います")
            data_list = [input_data]

    elif isinstance(input_data, list):
        print("💡 リストデータを処理します")
        data_list = input_data

    # 分かち書き
    return [tagger.parse(text).strip() for text in data_list]


# --- 3. 行ごと ROUGE → CSV 出力 ---

def compute_rouge_line_by_line(
    ref_txt="answer.txt",
    pred_txt="mt5_summarize.txt",
    output_csv="mt5_Rouge.csv"
):
    tagger = MeCab.Tagger(f"-O wakati {ipadic.MECAB_ARGS}")

    # 参照 & 生成文 読み込み
    with open(ref_txt, "r", encoding="utf-8") as r:
        refs = [l.strip() for l in r if l.strip()]

    with open(pred_txt, "r", encoding="utf-8") as p:
        preds = [l.strip() for l in p if l.strip()]

    if len(refs) != len(preds):
        raise ValueError(
            f"❌ 行数が一致しません\n参照: {len(refs)} 行\n生成: {len(preds)} 行"
        )

    # CSV 出力
    with open(output_csv, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["行番号", "ROUGE-1", "ROUGE-2", "ROUGE-L"])

        for i, (ref_line, pred_line) in enumerate(zip(refs, preds), start=1):
            ref_w = tagger.parse(ref_line).strip()
            pred_w = tagger.parse(pred_line).strip()

            pred_ids, ref_ids = convert_words_to_ids([pred_w], [ref_w])
            rouge = rouge_scorer.RougeScorer(
                ["rouge1", "rouge2", "rougeL"], use_stemmer=False
            )
            score = rouge.score(ref_ids[0], pred_ids[0])

            writer.writerow([
                i,
                score["rouge1"].fmeasure,
                score["rouge2"].fmeasure,
                score["rougeL"].fmeasure
            ])

    print(f"✅ 行ごとの ROUGE を '{output_csv}' に保存しました！")


# --- 4. 実行例 ---

compute_rouge_line_by_line(
    ref_txt="answer.txt",
    pred_txt="qwen1.5.txt",
    output_csv="qwem1.5_Rouge.csv"
)


✅ 行ごとの ROUGE を 'qwem1.5_Rouge.csv' に保存しました！
